# Transfer Learning
1. Introduction to Transfer Learning:

- Transfer learning is a machine learning technique where a pre-trained model developed for one task is reused as the starting point for a model on a second task.
- It leverages the knowledge gained while solving one problem and applies it to a different but related problem.
2. Why Transfer Learning?:

- Transfer learning saves time and computational resources since you start with a model that has already learned useful features from a large dataset.
- It often leads to better performance, especially when you have limited data for the specific task you want to solve.

##### We will have deeper understanding of transfer learning at the end of this notebook. So for now, we follow below steps:
1. Take pretrained model:- mobilenet v2, this model classifies 1000 classes(doesn't include any flower classification).
- The pretained model perfors terrible on the flower dataset(tf_flowers) as it was not trained for it's classification.
2. Freeze all the layers except the last one.
3. Train the last year for our flowers dataset - tf_flowers, (roses,tulips,sunflower,etc).
4. Get result.
5. Explaination of all the working

#### Import Relevant Libraries


In [2]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers

#### Download the flowers dataset

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin = dataset_url, cache_dir = '"C:/Users/satha/Downloads/datasets"', untar = True)

# Explaination done in tf_flowers project

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

# Explaination done in tf_flowers project

WindowsPath('/tmp/.keras/datasets/flower_photos')

In [7]:
# now we create a dictionary for all the flower image paths and corresponding labels

flower_image_dic = {
                        "roses" : list(data_dir.glob('roses/*')),
                        "daisy" : list(data_dir.glob('daisy/*')),
                        "dandelion" : list(data_dir.glob('dandelion/*')),
                        "sunflowers" : list(data_dir.glob('sunflowers/*')),
                        "tulips" : list(data_dir.glob('tulips/*'))
                    }

flower_labels_dic = {
                        'roses' : 0,
                        'daisy' : 1,
                        'dandelion': 2,
                        'sunflowers': 3,
                        'tulips' : 4
                    }

now we do the following:
1. read each file into numpy array via path by using: 
-  cv2.imread( str( file_path [index] ))
2. Resize all the image shapes into one value
3. store resized image array into a list
4. store corresponding label into another list

In [12]:
x , y =[], []
image_size = (224,224)
for flower_name, images in flower_image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, image_size)
        x.append(resized_img)
        y.append(flower_labels_dic[flower_name])
    

now we do the following:
1. convert x and y to numpy arrays
2. scale the data
3. split the data

In [14]:
x= np.array(x)
y=np.array(y)

In [15]:
x = x/255

In [27]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs,train_targets, test_targets = train_test_split(x,y,random_state=42)

At this stage if we feed this data into mobilenet v2 pretrained model, the model that has been trained on more than 1 million inputs , we will fail to classify these flowers(as it was not trained to classify into these flowers)

#### Pretrained model


In [19]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_last_layer = hub.KerasLayer(
    feature_extractor_model, 
    input_shape=(224,224,3), 
    trainable = False   # freeze layers
)

The 1st line gives the pretrained model without the last layer

#### Now we create our model

In [20]:
no_of_flowers = 5

model = tf.keras.Sequential([
                                pretrained_model_without_last_layer,

                                tf.keras.layers.Dense(no_of_flowers)
                            ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [21]:
model.compile(
    optimizer = 'adam',
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

note: 
1. We used loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True) instead of loss = 'sparse_categorical_crossentropy'
2. In our model we did not add any layer for softmax function, the above loss ensures it's been used externally.
3. This argument indicates that the model's output is provided in the form of logits. Logits are the raw, unnormalized scores produced by the model before converting them into probabilities using a softmax activation function. In other words, the model's output is assumed to be the pre-softmax values.

In [29]:
model.fit(train_inputs,train_targets, epochs = 5)

Epoch 1/5
86/86 [==============================] - 54s 545ms/step - loss: 0.8319 - accuracy: 0.6879
Epoch 2/5
86/86 [==============================] - 42s 485ms/step - loss: 0.4292 - accuracy: 0.8514
Epoch 3/5
86/86 [==============================] - 33s 387ms/step - loss: 0.3387 - accuracy: 0.8859
Epoch 4/5
86/86 [==============================] - 30s 347ms/step - loss: 0.2830 - accuracy: 0.9135
Epoch 5/5
86/86 [==============================] - 32s 377ms/step - loss: 0.2426 - accuracy: 0.9281


Now we test it on test dataset

In [30]:
model.evaluate(test_inputs, test_targets)

29/29 [==============================] - 11s 357ms/step - loss: 0.3441 - accuracy: 0.8791


[0.34413328766822815, 0.8790849447250366]

# Conclusion

We got accuracy of 88 percent

# Explaination of Transfer learning

1. How does changing only the last layer, makes the model learn about classification of flowers, even though it was not pretrained for it to begin with?
- The early layers of the model learn general features like edges and textures, which can be applied to various tasks, including flower classification. By adding a new output layer for your specific classes (e.g., roses, tulips), and training only this new layer while keeping the rest fixed, you reuse the valuable features learned by the pre-trained model. The initial layers capture basic features useful for many tasks, including distinguishing between different flowers. Essentially, you're using the model's prior knowledge and fine-tuning it for your unique classification task.

2. Arent the frezzed layer optimised to predict the 1000 classes meantioned in that pretrained model, then how can it predict flowers?
- The frozen layers are designed to extract general image features like edges, shapes, and textures, which are useful but not task-specific. Freezing them preserves their feature-extraction abilities. You're essentially letting the model fine-tune its final classification layer for your specific task while capitalizing on these pre-learned features.
- In MobileNetV2, mid-level features represent general visual patterns and textures from its original training, including shapes, textures, and colors relevant to various objects, including flowers. When fine-tuning for flower classification, you leverage the model's image understanding capabilities, even though it wasn't originally for flowers. Success comes from recognizing general visual patterns, textures, and shapes, not specific flower knowledge. So, while not initially designed for flowers, MobileNetV2's mid-level features adapt to recognize flower-related features effectively.

3. What are the features learned by these freezed layers?
- Low-Level Features:

   - Edges: The frozen layers capture essential edge information, crucial for distinguishing petal edges and flower boundaries.
   - Textures: These layers recognize simple textures found in flower petals, leaves, or backgrounds.
   - Colors: The model learns a variety of colors and color combinations, aiding in flower color identification.
- Mid-Level Features:

   - Geometric Patterns: These layers detect complex patterns, such as petal or leaf arrangements in different flowers.
   - Basic Shapes: They identify basic shapes that characterize specific types of flowers.